In [1]:
# %%
import pandas as pd
#import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
#import plotly.graph_objects as go
# 출력 제한 해제
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_seq_items', None)
pd.options.display.max_colwidth = 100

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
# %%
train = pd.read_csv('./data/train.csv')
test =  pd.read_csv('./data/test.csv')
trade = pd.read_csv('./data/international_trade.csv')

train.head()


,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0


In [3]:
# %%
time = pd.to_datetime(train['timestamp'].copy())

train['Date'] = pd.to_datetime(train['timestamp'])
train['week'] = train['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
train['day_name'] = train['Date'].dt.day_name()

train['year'] = train['timestamp'].apply(lambda x : int(x[0:4]))
train['month'] = train['timestamp'].apply(lambda x : int(x[5:7]))
train['day'] = train['timestamp'].apply(lambda x : int(x[8:10]))

In [4]:
# %%
import datetime
import holidays

# %%
#train = train[(train['supply(kg)'] != 0) & (train['supply(kg)'] != 0)] # 이거차이도 큼!!! 거의 rmse 1200에서 680으로 줄어듦. 근데 이건 validation에만이고 test는 이거하면 정반대로됨 ㅠ...

# %%
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
test_re = test.copy()

test_re['year'] = test_re['timestamp'].apply(lambda x : int(x[0:4]))
test_re['month'] = test_re['timestamp'].apply(lambda x : int(x[5:7]))
test_re['day'] = test_re['timestamp'].apply(lambda x : int(x[8:10]))
test_re['Date'] = pd.to_datetime(test_re['timestamp'])
test_re['week'] = test_re['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
test_re['day_name'] = test_re['Date'].dt.day_name()


# %%
test_re.head()

,ID,timestamp,item,corporation,location,year,month,day,Date,week,day_name
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,4,2023-03-04,9,Saturday
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,5,2023-03-05,9,Sunday
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,6,2023-03-06,10,Monday
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,7,2023-03-07,10,Tuesday
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,8,2023-03-08,10,Wednesday


In [5]:
# %%
from sklearn.preprocessing import LabelEncoder
train_x = train.drop(columns=['ID', 'timestamp', 'supply(kg)', 'price(원/kg)','Date'])
train_y = train['price(원/kg)']
test_x = test_re.drop(columns=['ID', 'timestamp','Date'])
qual_col = ['item', 'corporation', 'location','day_name']

train_x = pd.get_dummies(train_x, columns=qual_col, prefix=qual_col)
test_x = pd.get_dummies(test_x, columns=qual_col, prefix=qual_col)

In [6]:
train_x = pd.concat([train_x, train_y], axis=1)

In [7]:
train_x.head()

,week,year,month,day,item_BC,item_CB,item_CR,item_RD,item_TG,corporation_A,corporation_B,corporation_C,corporation_D,corporation_E,corporation_F,location_J,location_S,day_name_Friday,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday,price(원/kg)
0,1,2019,1,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0.0
1,1,2019,1,2,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0.0
2,1,2019,1,3,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1728.0
3,1,2019,1,4,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1408.0
4,1,2019,1,5,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1250.0


In [8]:
test_x.head() #day_name 일치

,year,month,day,week,item_BC,item_CB,item_CR,item_RD,item_TG,corporation_A,corporation_B,corporation_C,corporation_D,corporation_E,corporation_F,location_J,location_S,day_name_Friday,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday
0,2023,3,4,9,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,2023,3,5,9,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0
2,2023,3,6,10,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0
3,2023,3,7,10,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0
4,2023,3,8,10,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1


In [21]:
# import pycaret regression and init setup
from pycaret.regression import *

s = setup(train_x, target = 'price(원/kg)', 
        session_id = 123, 
        fold = 10, 
        normalize = True, 
        normalize_method = 'minmax',
        train_size=0.8)

# Best models 선택 (예: 상위 3개 모델 선택)
top5_models = compare_models(n_select=5)


,Description,Value
0,Session id,123
1,Target,price(원/kg)
2,Target type,Regression
3,Original data shape,"(59397, 25)"
4,Transformed data shape,"(59397, 25)"
5,Transformed train set shape,"(47517, 25)"
6,Transformed test set shape,"(11880, 25)"
7,Numeric features,24
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,306.1269,730314.5152,854.3318,0.8236,2.1566,0.1877,1.2860
lightgbm,Light Gradient Boosting Machine,404.4009,787864.5216,887.2354,0.8098,3.5231,0.2552,0.3170
xgboost,Extreme Gradient Boosting,416.0808,794264.1750,890.7374,0.8084,3.7132,0.2447,0.0970
catboost,CatBoost Regressor,428.3602,804003.0424,896.2687,0.8059,3.7622,0.2498,1.0100
et,Extra Trees Regressor,316.9103,832598.2251,912.2040,0.7988,2.0480,0.1993,1.4410
knn,K Neighbors Regressor,492.4061,1152218.7188,1072.8099,0.7219,2.6017,0.3368,0.1630
gbr,Gradient Boosting Regressor,604.4963,1191365.2540,1091.1368,0.7124,4.2219,0.3291,0.6830
dt,Decision Tree Regressor,370.9626,1342224.8938,1157.6989,0.6762,2.0815,0.2556,0.0480
ridge,Ridge Regression,1063.7833,2361673.3306,1536.3330,0.4300,4.8966,0.5791,0.0320
br,Bayesian Ridge,1063.7518,2361671.4531,1536.3320,0.4300,4.8969,0.5790,0.0380


In [ ]:
tuned_top5 = [tune_model(i) for i in top5_models]

blended = blend_models(estimator_list = tuned_top5,
                       fold = 10,
                       optimize='RMSE',
                       )

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,01:01:59
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [ ]:
final_model = finalize_model(blended)

In [ ]:
predictions = predict_model(final_model, data = pd.DataFrame(test_x))

In [ ]:
result = pd.concat([test_re, predictions['prediction_label']], axis = 1)

result['answer'] = result['prediction_label'].copy()

del result['timestamp']
del result['item']
del result['corporation']
del result['location']
del result['year']
del result['month']
del result['day']
del result['Date']
del result['week']
del result['day_name']
del result['prediction_label']

In [ ]:
for i in range(len(result)):
    if (i-1) % 7 == 0: # 1, 8, 15, 22
        result.iloc[i,-1] = 0

result.head(20)

In [ ]:
result.to_csv('result3_3_tuned.csv', index = False)